# HOS 

In [ ]:
import _Library_HOS
from Classe_sismogramma_v3 import ClasseDataset
import pandas as pd

# ATTENTO prima di usate get_onset, 1) Fallo su quelle a 200 HZ
                                #   2) PRIMA devi fare filtro passa alto (sui 3-5 Hz)
# parametri di metodo HOS DA OTTIMIZZARE "SUL POLLINO" (però siamo a 100 HZ!!!! poi fai anche test su quelle già picked di DETECT?)
# Scrivi il codice che usi in un unico file e condividi!

hd = "/home/silvia/Desktop/Data/Pollino_All/Pollino_All_data_extended.hdf5"
cs = "/home/silvia/Desktop/Data/Pollino_All/Pollino_All_metadata_extended.csv"

D = ClasseDataset()
D.leggi_custom_dataset(hd,cs)
D.demean()

# sig = _Library_HOS.freq_filter(D.sismogramma[7], 200, 5, type_filter= "highpass")
# onset_1, diff, onset_2 = _Library_HOS.get_onset(sig, 40, threshold=0.1, statistics= _Library_HOS.S_6)
# print(onset_1, onset_2)


In [ ]:
import _Library_HOS
sig = _Library_HOS.freq_filter(D.sismogramma[7], 200, 5, type_filter= "highpass")
onset_1, diff, onset_2 = _Library_HOS.get_onset(sig, 40, threshold=0.1, statistics= _Library_HOS.S_6)
print(onset_1, onset_2)

# TEST

Mi sa che devo cambiare il programma... da get_hos per fargli fare le cose in parallelo. per ora fa una traccia alla volta..